In [80]:
import pandas as pd

def count_exons (file):
    df = pd.read_csv(file, sep='\t', header = None)
    starts = []
    ends = []
    coor = df.iloc(axis=1)[3:5]
    coor.columns = ['start', 'end']
    for i in range (coor.shape[0]):
        starts.append(int(coor.loc[i][0]))
        ends.append(int(coor.loc[i][1]))
    starts = sorted (starts)
    ends = sorted (ends)
    prev_end = 0
    sum = 0
    for i in range (len(starts)):
        if starts[i] > prev_end:
            sum += ends[i] - starts[i] + 1
        else:
            sum += ends[i] - prev_end
        prev_end = ends[i]
    return(sum)
print(f'GNG4: {count_exons("GNG4_exons.tsv")}')
print(f'SPRR4: {count_exons("SPRR4_exons.tsv")}')

GNG4: 5401
SPRR4: 733


In [84]:
import pandas as pd
import numpy as np

df = pd.read_csv('TCGA-COAD_cancer_normal.tsv', sep = '\t', index_col = 0)
df = df.loc[df.min(axis = 1) > 0]
psd_sample = np.log2(df).mean(axis=1)

for i in df.columns:
    df[i] = np.log2(df[i]) - psd_sample #перевод логарифмов, вычитание логарифмов = деление

ans = 2**df.median(axis = 0)#взяла медиану, возвела в степень 2, чтобы получить не  логарифм от норм множителей, а сами норм множители
for i,j in zip(df.columns,ans):
    print(i,": ",j)


TCGA-A6-2682-01A :  0.35219656418871853
TCGA-A6-2683-01A :  0.3943908614475186
TCGA-A6-2685-01A :  0.7305734402506026
TCGA-A6-2686-01A :  1.6613807884976424
TCGA-A6-5662-01A :  1.6000283761172205
TCGA-A6-5667-11A :  1.4831361575696305
TCGA-AA-3489-11A :  1.280469709326452
TCGA-AA-3496-11A :  0.9243427405663923
TCGA-AA-3511-11A :  1.5930679895831408
TCGA-AA-3514-11A :  1.3499769758243516


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('TCGA-COAD_cancer_normal.tsv', sep = '\t', index_col = 0)
df = df.loc[df.min(axis = 1) > 0]
psd_sample = np.log2(df).mean(axis=1)

for i in df.columns:
    df[i] = np.log2(df[i]) - psd_sample 
    
ans = 2**df.median(axis = 0)

for i,j in zip(df.columns,ans):
    print(i,": ",j)


In [85]:
factors = [0.35219656, 0.39439086, 0.73057344, 1.66138079, 
1.60002838, 1.48313616, 1.28046971, 0.92434274, 1.59306799, 1.34997698]

df = pd.read_csv("TCGA-COAD_cancer_normal.tsv", sep="\t", index_col = 0).sort_index()
gl = pd.read_csv("gene_lengths.tsv", sep="\t", index_col = 0).sort_index()

RPM = df.div(df.sum(axis = 0), axis = 1) * 1e+6 
DESeq2_RPM = RPM.div(factors, axis = 1)
DESeq2_RPKM = RPM.div(gl["Length"], axis = 0) * 1000

DESeq2_RPKM['Sum'] = DESeq2_RPKM.sum(axis=1)
DESeq2_RPKM = DESeq2_RPKM.sort_values(by = "Sum", ascending = False)
DESeq2_RPKM.iloc[:10, [10]]

,Sum
Gene,
MT-CO2,72102.322291
MT-CO1,58858.580222
MT-CO3,56618.695898
MT-ATP6,45204.439791
MT-ND4,43893.627928
MT-RNR2,38238.690293
MT-ND3,31902.368503
FTL,30938.192674
TMSB10,30700.089507


In [86]:
factors = [0.35219656, 0.39439086, 0.73057344, 1.66138079, 
1.60002838, 1.48313616, 1.28046971, 0.92434274, 1.59306799, 1.34997698]

df = pd.read_csv("TCGA-COAD_cancer_normal.tsv", sep="\t", index_col=0).sort_index() # сортируем, чтобы при делении на длину генов строчки в таблицах совпадали
gl = pd.read_csv("gene_lengths.tsv", sep="\t", index_col=0).sort_index()


RPM = df.div(df.sum(axis=0), axis = 1) * 1e+6
dsq2 = RPM.div(factors, axis = 1) 
dsq2 = RPM.div(gl["Length"], axis = 0) * 1000

dsq2 = dsq2[dsq2.min(axis = 1) > 0]

dsq2['std'] = dsq2.std(axis=1)
dsq2 = dsq2.sort_values(by = "std")
dsq2.iloc[:10, [10]]

,std
Gene,
HELLPAR,0.000935
LINC01122,0.004313
SPECC1L-ADORA2A,0.004686
RP3-412A9.17,0.005074
CTA-228A9.4,0.006640
chr22-38_28785274-29006793.1,0.006670
DNAH11,0.007936
TBC1D29,0.008045
ZNF259P1,0.010230


In [87]:
from scipy.stats import t
import numpy as np
import pandas as pd

factors = [0.35219656, 0.39439086, 0.73057344, 1.66138079, 
1.60002838, 1.48313616, 1.28046971, 0.92434274, 1.59306799, 1.34997698]

df = pd.read_csv("TCGA-COAD_cancer_normal.tsv", sep="\t", index_col=0)
gl = pd.read_csv("gene_lengths.tsv", sep="\t", index_col=0).sort_index()
df.sort_index()
gl.sort_index()

RPM = df.div(df.sum(axis=0), axis = 1) * 1e+6
dsq2 = RPM.div(factors, axis = 1) 
dsq2 = RPM.div(gl["Length"], axis = 0) * 1000

dsq2 = dsq2[dsq2.min(axis = 1) > 0]

mean_tumor = np.mean(dsq2.iloc[:, :5], axis = 1)
mean_health = np.mean(dsq2.iloc[:, 5:], axis = 1)
std_tumor = dsq2.iloc[:, :5].std(axis = 1)
std_health = dsq2.iloc[:, 5:].std(axis = 1)

dsq2['t'] = abs((mean_tumor - mean_health)/(std_tumor**2/5 + std_health**2/5)**0.5)
dsq2 = dsq2[dsq2['t'] > abs(t.ppf(0.05/2, df =  8))]
dsq2 = dsq2.sort_values(by = 't', ascending = False)
dsq2.iloc[:10, [-1]]

,t
Gene,
PRKACB,17.305850
GUCA2A,13.908860
C17orf96,13.440390
ABCF2,11.430203
ATP5A1,11.379903
MS4A2,10.628818
SCP2,10.260262
BCAP31,9.996616
NTMT1,9.893826
